In [4]:
from ruamel import yaml
import src.blocks as blocks
from src import Block
from src import merge
import copy
from collections import OrderedDict
from jinja2 import Environment, FileSystemLoader

In [5]:
# Initialize yaml reader
ryaml = yaml.YAML()

In [6]:
# Read configuration
with open("config.yaml", "r") as f:
    config = ryaml.load(f)

config


{'some_ints': {'a': 2, 'b': 10, 'c': 4}, 'a_float': {'d': 0.5}}

In [7]:
# Read master
with open("master.yaml", "r") as f:
    try:
        master = ryaml.load(f)
    except yaml.YAMLError as e:
        print(
            "It seems that you have duplicate keys in your master file. Please ensure that no block"
            " is being called twice with the same name in a given scope. If that's the case, please"
            " append '__x' to the end of the block name, where x corresponds to the xth repetition"
            " of the block."
        )
        print(e)
        exit(1)

In [8]:
for gen in sorted(master.keys()):
    print("Now browsing generation: " + gen)
    for block in master[gen]['script']:
        print("Now browsing block: " + block)
        print(master[gen]['script'][block])
    print()

Now browsing generation: gen1
Now browsing block: multiply
{'args': ['b', 'c'], 'output': 'bc'}
Now browsing block: add
{'args': ['a', 'bc'], 'output': 'a_bc'}
Now browsing block: factorial
{'args': 'a_bc', 'output': 'fact_a_bc'}

Now browsing generation: gen2
Now browsing block: add_power
{'args': ['b', 'c', 'c'], 'output': 'bc_c'}
Now browsing block: multiply
{'args': ['a', 'bc_c'], 'output': 'a_bc_c'}
Now browsing block: add_power__2
{'args': ['c', 'c', 'd'], 'output': 'c_c_d'}
Now browsing block: add
{'args': ['a_bc_c', 'c_c_d'], 'output': 'a_bc_c_c_d'}
Now browsing block: multiply__2
{'args': ['fact_a_bc', 'a_bc_c_c_d'], 'output': 'result'}



In [9]:
# Isn't this function just a special case of merge? Probably
def generate_main(script, config, dict_blocks, save = None, dict_imports_merge = None):

    # Generate header
    header_str = "def main():\n"

    # Declare parameters
    parameters_str = "\t# Declare parameters\n"
    for param, value in config.items():
        parameters_str += "\t" + param + " = " + str(value) + "\n"

    # Declare blocks
    blocks_str = "\n\t# Declare blocks\n"
    for block, dict_block in script.items():
        # Get block arguments
        l_args = dict_block["args"]
        if not isinstance(l_args, list):
            l_args = [l_args]
        # Get block outputs
        l_outputs = dict_block["output"]
        if not isinstance(l_outputs, list):
            l_outputs = [l_outputs]
        # Get true block in case of repeaded key
        if '__' in block:
            true_block = block.split('__')[0]
        else:
            true_block = block
        # Write blocks string
        blocks_str += "\t" + dict_blocks[true_block].get_assignation_call_str(l_args, l_outputs) + "\n"

    # Save output # ! These saves should be blocks
    if save is not None:
        if dict_imports_merge is not None:
            save_str = "\n\t# Save output\n"
            for output, save_method in save.items():
                if save_method == 'npy':
                    save_str += f"\t{dict_imports_merge['numpy']}.save('{output}', {output})\n"
                elif save_method == 'pickle':
                    save_str += f"\twith open('{output}', 'wb') as f:\n"
                    save_str += f"\t\t{dict_imports_merge['pickle']}.dump({output}, f)\n"
        else:
            raise ValueError("dict_imports_merge must be provided to save output")
    else:
        save_str = ""           

    main_str = header_str + parameters_str + blocks_str + save_str

    # Replace tabs by spaces to prevent inconsistent indentation
    main_str = main_str.replace("\t", "    ")

    return main_str

In [10]:
# Generate generations from template
environment = Environment(loader=FileSystemLoader("templates/"))
template = environment.get_template("default_template.txt")

for gen in sorted(master.keys()):
    file_path = f"{gen}.py"
    print("Now browsing generation: " + gen)
    print("\n")
    dict_blocks = OrderedDict()
    
    # Get set of new (merged) blocks
    set_new_blocks = set()
    if "new_blocks" in master[gen]:
        set_new_blocks.update(master[gen]["new_blocks"].keys())
        
    # Get all blocks (except new blocks)
    for block in master[gen]["script"]:
        if '__' in block:
            # Don't want to declare twice the same block
            continue
        if block not in set_new_blocks:
            dict_blocks[block] = getattr(blocks, block)

    # Get blocks used for new blocks
    for new_block in set_new_blocks:
        for block in master[gen]["new_blocks"][new_block]["blocks"]:
            # ! I need to find a way to do this iteratively, as there might be several levels of new blocks
            dict_blocks[block] = getattr(blocks, block)

    # Merge all imports
    dict_imports_merge = merge.merge_imports(list(dict_blocks.values()))

    # Add the imports for save methods
    if "save" in master[gen]:
        for save_method in master[gen]["save"].values():
            if save_method == "npy" and "numpy" not in dict_imports_merge:
                dict_imports_merge["numpy"] = "np"
            if save_method == "pkl" and "pickle" not in dict_imports_merge:
                dict_imports_merge["pickle"] = "pickle"

    # Get string imports
    str_imports = Block.get_external_l_imports_str(dict_imports_merge)

    # Build new blocks
    if "new_blocks" in master[gen]:
        for new_block_name, new_block in master[gen]["new_blocks"].items():

            # Update parameters of each block to match the merged block specification
            l_blocks = []
            for block in new_block["blocks"]:
                block_to_update = copy.deepcopy(dict_blocks[block])
                l_params = new_block["blocks"][block]["params"]
                l_outputs = new_block["blocks"][block]["output"]
                block_to_update.set_parameters_names(l_params)
                block_to_update.set_outputs_names(l_outputs)
                l_blocks.append(block_to_update)

            # Get the dict of final output (with undefined type for now)
            output_final = new_block['output']
            if not isinstance(output_final, list):
                output_final = [output_final]
            dict_outputs_final = OrderedDict([(output, None) for output in output_final])
            
            # Find the type of output
            for block in l_blocks:
                for output in block.dict_output:
                    if output in dict_outputs_final:
                        dict_outputs_final[output] = block.dict_output[output]
            
            # Raise an error if some outputs are not defined
            if None in dict_outputs_final.values():
                raise ValueError("Some outputs are not defined")

            new_block_function = merge.merge_blocks(
                l_blocks,
                new_block_name,
                docstring=new_block["docstring"],
                dict_output=dict_outputs_final,
            )
            
            dict_blocks[new_block_name] = new_block_function

    # Get the dictionnary of block strings
    dict_blocks_str = {k: v.get_str() for k, v in dict_blocks.items()}

    # Get corresponding block string
    str_blocks = "\n".join([f"{k}" for k in dict_blocks_str.values()])

    # Handle output save
    if "save" in master[gen]:
        save = master[gen]["save"]
    else:
        save = None

    # Write main script
    str_main = generate_main(
        master[gen]["script"], config[gen], dict_blocks, save, dict_imports_merge
    )

    # Render template
    content = template.render(
        imports=str_imports,
        blocks=str_blocks,
        main=str_main,
    )
    with open(file_path, mode="w", encoding="utf-8") as file:
        file.write(content)

Now browsing generation: gen1




KeyError: 'gen1'